# This notebook was run on Kaggle

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers


In [ ]:
# Get the file paths
h5_files = '/kaggle/input/inception-weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'



In [ ]:
# Define paths
train_dir = "/kaggle/input/cropped-pins-face-recognition/train"
val_dir = "/kaggle/input/cropped-pins-face-recognition/val"
test_dir = "/kaggle/input/cropped-pins-face-recognition/test"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),batch_size=64,class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(val_dir,target_size=(150, 150),batch_size=64,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,target_size=(150, 150),batch_size=64,class_mode='categorical')

Found 12225 images belonging to 105 classes.
Found 2632 images belonging to 105 classes.
Found 2675 images belonging to 105 classes.


In [ ]:
base_model = tf.keras.applications.InceptionV3(weights=h5_files, include_top=False, input_shape=(150, 150, 3))
# Freeze InceptionV3 layers
for layer in base_model.layers[:-1]:  # Unfreeze last 10 layers
    layer.trainable = True

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = layers.Dense(105, activation='softmax')(x)
model = models.Model(inputs=base_model.input, outputs=predictions)



# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator,epochs=70,validation_data=validation_generator)


Epoch 1/70
  1/192 ━━━━━━━━━━━━━━━━━━━━ 4:04:39 77s/step - accuracy: 0.0000e+00 - loss: 18.8877

W0000 00:00:1713465016.403780     108 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 579ms/step - accuracy: 0.0136 - loss: 9.7507

W0000 00:00:1713465133.613882     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


192/192 ━━━━━━━━━━━━━━━━━━━━ 202s 657ms/step - accuracy: 0.0137 - loss: 9.7341 - val_accuracy: 0.0198 - val_loss: 5.7079
Epoch 2/70


W0000 00:00:1713465142.100720     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


192/192 ━━━━━━━━━━━━━━━━━━━━ 95s 475ms/step - accuracy: 0.0230 - loss: 4.3601 - val_accuracy: 0.0289 - val_loss: 14.6907
Epoch 3/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 462ms/step - accuracy: 0.0387 - loss: 4.0608 - val_accuracy: 0.0498 - val_loss: 4.2911
Epoch 4/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 95s 476ms/step - accuracy: 0.0622 - loss: 4.4612 - val_accuracy: 0.0992 - val_loss: 3.8324
Epoch 5/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 462ms/step - accuracy: 0.0957 - loss: 3.5888 - val_accuracy: 0.1121 - val_loss: 3.8328
Epoch 6/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 460ms/step - accuracy: 0.1263 - loss: 3.4743 - val_accuracy: 0.0976 - val_loss: 4.0920
Epoch 7/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 90s 455ms/step - accuracy: 0.1629 - loss: 3.2709 - val_accuracy: 0.1839 - val_loss: 3.5439
Epoch 8/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 92s 463ms/step - accuracy: 0.2042 - loss: 3.2323 - val_accuracy: 0.1797 - val_loss: 3.4282
Epoch 9/70
192/192 ━━━━━━━━━━━━━━━━━━━━ 91s 460ms/step - accuracy: 0.2494 - loss: 2.9499 - va

In [ ]:
# Save the trained model
model.save("/kaggle/working/my_model.h5")


In [ ]:
# Save the trained model weights
model.save_weights("/kaggle/working/my_model_weights.weights.h5")

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


42/42 ━━━━━━━━━━━━━━━━━━━━ 40s 966ms/step - accuracy: 0.8574 - loss: 0.8419
Test Loss: 0.886438250541687
Test Accuracy: 0.8542056083679199


W0000 00:00:1713471861.477160     108 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
